# Imports

In [1]:
import requests
import scipy
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sb
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, norm
import matplotlib.pyplot as plt 
sb.set() 
from math import sqrt
from tensorflow import keras
from statsmodels.tools import categorical
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import datetime, timedelta
from keras.models import load_model
from joblib import dump, load

# Select variable to predict 
`String prediction_variable`

In [4]:
# Main variable = 'MainIncoming_kW'
prediction_variable = 'MainIncoming_kW'

# Remove Units

In [11]:
data = pd.read_csv('Training_Data.csv')
# remove units from length column and convert to float
data['Timestamp'] = data['Timestamp'].str.replace(' Singapore', '').astype(str)
data['MainIncoming_kW'] = data['MainIncoming_kW'].str.replace('kW', '').astype(float)
data['CHWRT'] = data['CHWRT'].str.replace('°C', '').astype(float)
data['CWRT'] = data['CWRT'].str.replace('°C', '').astype(float)
data['CHWST'] = data['CHWST'].str.replace('°C', '').astype(float)
data['CWST'] = data['CWST'].str.replace('°C', '').astype(float)
data['CHW Flow'] = data['CHW Flow'].str.replace('L/s', '').astype(float)
data['CDW Flow'] = data['CDW Flow'].str.replace('L/s', '').astype(float)
data['heat_in'] = data['heat_in'].str.replace('_RT', '').astype(float)
data['heat_out'] = data['heat_out'].str.replace('_RT', '').astype(float)
data['lift'] = data['lift'].str.replace('Δ°C', '').astype(float)
data['chwdt'] = data['chwdt'].str.replace('Δ°C', '').astype(float)
data['cwdt'] = data['cwdt'].str.replace('Δ°C', '').astype(float)
data['DemoSite Demo avgHum'] = data['DemoSite Demo avgHum'].str.replace('%RH', '').astype(float)
data['DemoSite Demo avgTemp'] = data['DemoSite Demo avgTemp'].str.replace('°C', '').astype(float)
data['DemoSite Demo Calculated Wet Bulb'] = data['DemoSite Demo Calculated Wet Bulb'].str.replace('°C', '').astype(float)
data

,Timestamp,MainIncoming_kW,CHWRT,CWRT,CHWST,CWST,CHW Flow,CDW Flow,heat_in,heat_out,lift,chwdt,cwdt,DemoSite Demo avgHum,DemoSite Demo avgTemp,DemoSite Demo Calculated Wet Bulb
0,2023-01-01T00:00:00+08:00,22.484142,9.954571,24.291482,9.510906,24.272862,36.892751,45.632243,5.865666,0.538440,14.780576,0.443665,0.018620,81.00,25.0,22.466977
1,2023-01-01T00:15:00+08:00,39.776803,9.797504,25.524160,8.399437,24.342649,36.846134,45.655656,19.983421,34.264709,17.124722,1.398067,1.181510,81.25,25.0,22.501307
2,2023-01-01T00:30:00+08:00,28.203817,10.344149,24.502372,9.410065,24.244196,36.897374,45.643374,13.142357,7.457039,15.092308,0.934084,0.258177,81.50,25.0,22.535621
3,2023-01-01T00:45:00+08:00,31.287828,9.553039,25.054075,8.588733,24.517560,36.885815,45.633395,13.440550,15.671155,16.465342,0.964306,0.536515,81.75,25.0,22.569920
4,2023-01-01T01:00:00+08:00,34.554298,10.371635,24.875535,9.201972,24.294594,36.871562,45.627637,17.309697,16.814469,15.673563,1.169663,0.580941,82.00,25.0,22.604205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14587,2023-06-01T22:45:00+08:00,38.565986,10.462083,26.667786,8.869298,25.405509,37.133546,40.760041,23.069997,38.205330,17.798487,1.592785,1.262276,77.25,29.0,23.569493
14588,2023-06-01T23:00:00+08:00,38.752105,10.248372,26.775094,8.878301,25.512510,37.125840,40.747375,20.782590,38.375578,17.896793,1.370071,1.262583,77.00,29.0,23.569493
14589,2023-06-01T23:15:00+08:00,39.042879,10.363150,27.134521,8.872899,25.936682,37.153195,40.741234,22.127904,37.013031,18.261622,1.490250,1.197839,77.00,29.0,23.569493
14590,2023-06-01T23:30:00+08:00,39.149259,10.327005,27.192183,8.860048,25.884210,37.126226,40.775393,21.901174,40.334384,18.332135,1.466957,1.307973,77.00,29.0,23.569493


## Check data type (float64)

In [12]:
print(data.dtypes)

Timestamp                             object
MainIncoming_kW                      float64
CHWRT                                float64
CWRT                                 float64
CHWST                                float64
CWST                                 float64
CHW Flow                             float64
CDW Flow                             float64
heat_in                              float64
heat_out                             float64
lift                                 float64
chwdt                                float64
cwdt                                 float64
DemoSite Demo avgHum                 float64
DemoSite Demo avgTemp                float64
DemoSite Demo Calculated Wet Bulb    float64
dtype: object


# Extract Individual Dates

In [13]:
data['Timestamp'] =pd.to_datetime(data['Timestamp'], format="%Y-%m-%dT%H:%M:%S%z")
# convert the timestamp to the corresponding day of the week
data['Day of Week'] = data['Timestamp'].dt.strftime('%A')
# Convert datetime to weekday numbers (Monday=1, Tuesday=2, etc.)
data['DayNumber'] = data['Timestamp'].dt.weekday
# add 1 to get days starting from 1
data['DayNumber'] = data['DayNumber'] + 1

data['Year'] = data['Timestamp'].dt.year
data['Month'] = data['Timestamp'].dt.month
data['Day'] = data['Timestamp'].dt.day
data['Hour'] = data['Timestamp'].dt.hour
data['Minute'] = data['Timestamp'].dt.minute
data

,Timestamp,MainIncoming_kW,CHWRT,CWRT,CHWST,CWST,CHW Flow,CDW Flow,heat_in,heat_out,...,DemoSite Demo avgHum,DemoSite Demo avgTemp,DemoSite Demo Calculated Wet Bulb,Day of Week,DayNumber,Year,Month,Day,Hour,Minute
0,2023-01-01 00:00:00+08:00,22.484142,9.954571,24.291482,9.510906,24.272862,36.892751,45.632243,5.865666,0.538440,...,81.00,25.0,22.466977,Sunday,7,2023,1,1,0,0
1,2023-01-01 00:15:00+08:00,39.776803,9.797504,25.524160,8.399437,24.342649,36.846134,45.655656,19.983421,34.264709,...,81.25,25.0,22.501307,Sunday,7,2023,1,1,0,15
2,2023-01-01 00:30:00+08:00,28.203817,10.344149,24.502372,9.410065,24.244196,36.897374,45.643374,13.142357,7.457039,...,81.50,25.0,22.535621,Sunday,7,2023,1,1,0,30
3,2023-01-01 00:45:00+08:00,31.287828,9.553039,25.054075,8.588733,24.517560,36.885815,45.633395,13.440550,15.671155,...,81.75,25.0,22.569920,Sunday,7,2023,1,1,0,45
4,2023-01-01 01:00:00+08:00,34.554298,10.371635,24.875535,9.201972,24.294594,36.871562,45.627637,17.309697,16.814469,...,82.00,25.0,22.604205,Sunday,7,2023,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14587,2023-06-01 22:45:00+08:00,38.565986,10.462083,26.667786,8.869298,25.405509,37.133546,40.760041,23.069997,38.205330,...,77.25,29.0,23.569493,Thursday,4,2023,6,1,22,45
14588,2023-06-01 23:00:00+08:00,38.752105,10.248372,26.775094,8.878301,25.512510,37.125840,40.747375,20.782590,38.375578,...,77.00,29.0,23.569493,Thursday,4,2023,6,1,23,0
14589,2023-06-01 23:15:00+08:00,39.042879,10.363150,27.134521,8.872899,25.936682,37.153195,40.741234,22.127904,37.013031,...,77.00,29.0,23.569493,Thursday,4,2023,6,1,23,15
14590,2023-06-01 23:30:00+08:00,39.149259,10.327005,27.192183,8.860048,25.884210,37.126226,40.775393,21.901174,40.334384,...,77.00,29.0,23.569493,Thursday,4,2023,6,1,23,30


In [14]:
data.drop(['Timestamp','Year','Month','Day','Day of Week'],axis=1, inplace=True)

# Rearrange order of columns
### Last column is the variable to be predicted

In [15]:
prediction_column = data.columns.get_loc(prediction_variable) # 0-based
if (prediction_variable == MainIncoming_kW):
    data = data.iloc[:, np.r_[prediction_column + 1:len(data.columns), prediction_column]]
else:
    data = data.iloc[:, np.r_[prediction_column + 1:len(data.columns), prediction_column]]
data

0


,CHWRT,CWRT,CHWST,CWST,CHW Flow,CDW Flow,heat_in,heat_out,lift,chwdt,cwdt,DemoSite Demo avgHum,DemoSite Demo avgTemp,DemoSite Demo Calculated Wet Bulb,DayNumber,Hour,Minute,MainIncoming_kW
0,9.954571,24.291482,9.510906,24.272862,36.892751,45.632243,5.865666,0.538440,14.780576,0.443665,0.018620,81.00,25.0,22.466977,7,0,0,22.484142
1,9.797504,25.524160,8.399437,24.342649,36.846134,45.655656,19.983421,34.264709,17.124722,1.398067,1.181510,81.25,25.0,22.501307,7,0,15,39.776803
2,10.344149,24.502372,9.410065,24.244196,36.897374,45.643374,13.142357,7.457039,15.092308,0.934084,0.258177,81.50,25.0,22.535621,7,0,30,28.203817
3,9.553039,25.054075,8.588733,24.517560,36.885815,45.633395,13.440550,15.671155,16.465342,0.964306,0.536515,81.75,25.0,22.569920,7,0,45,31.287828
4,10.371635,24.875535,9.201972,24.294594,36.871562,45.627637,17.309697,16.814469,15.673563,1.169663,0.580941,82.00,25.0,22.604205,7,1,0,34.554298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14587,10.462083,26.667786,8.869298,25.405509,37.133546,40.760041,23.069997,38.205330,17.798487,1.592785,1.262276,77.25,29.0,23.569493,4,22,45,38.565986
14588,10.248372,26.775094,8.878301,25.512510,37.125840,40.747375,20.782590,38.375578,17.896793,1.370071,1.262583,77.00,29.0,23.569493,4,23,0,38.752105
14589,10.363150,27.134521,8.872899,25.936682,37.153195,40.741234,22.127904,37.013031,18.261622,1.490250,1.197839,77.00,29.0,23.569493,4,23,15,39.042879
14590,10.327005,27.192183,8.860048,25.884210,37.126226,40.775393,21.901174,40.334384,18.332135,1.466957,1.307973,77.00,29.0,23.569493,4,23,30,39.149259


# (Optional) Drop all weekend data

In [ ]:
weekend_condition = data['DayNumber'] >= 6
data_filtered = data.drop(data[weekend_condition].index)
data_filtered

# Preparing Training Data

In [8]:
# Slice data into sequences
seq_len = 24*3*4 #3days in 15min interval
data_arr=[]
for i in range(len(data)-seq_len+1):
    data_slice = data[i:i+seq_len:]
    data_arr.append(data_slice)
data_arr = np.asarray(data_arr).astype('float32')
print(len(data_arr))

14305


# Split Data

In [9]:
# Split data
train = data_arr[:11444]
test_output = np.asarray(data)[11732:]
train_x, train_y = train[:,:-1,:-1], train[:,-1,-1]
test = data_arr[11732:]
test_x, test_y = test[:,:-1,:-1], test[:,-1,-1]
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

# Construct the tensors
# Reshape input to be 3D [samples, timesteps, features]
train_x = train_x.reshape((train_x.shape[0], seq_len-1, train_x.shape[2]))
test_x = test_x.reshape((test_x.shape[0], seq_len-1, test_x.shape[2]))
print(train_x)
print(test_x)

(11444, 287, 17)
(11444,)
(2573, 287, 17)
(2573,)
[[[ 9.954572  24.291481   9.510906  ...  7.         0.         0.       ]
  [ 9.797504  25.52416    8.399438  ...  7.         0.        15.       ]
  [10.344149  24.502373   9.410065  ...  7.         0.        30.       ]
  ...
  [10.588791  26.35831    8.905826  ...  2.        23.         0.       ]
  [10.4446745 26.415445   8.897853  ...  2.        23.        15.       ]
  [10.410767  26.386478   8.844352  ...  2.        23.        30.       ]]

 [[ 9.797504  25.52416    8.399438  ...  7.         0.        15.       ]
  [10.344149  24.502373   9.410065  ...  7.         0.        30.       ]
  [ 9.553039  25.054075   8.588733  ...  7.         0.        45.       ]
  ...
  [10.4446745 26.415445   8.897853  ...  2.        23.        15.       ]
  [10.410767  26.386478   8.844352  ...  2.        23.        30.       ]
  [10.405317  26.249859   8.801012  ...  2.        23.        45.       ]]

 [[10.344149  24.502373   9.410065  ...  7.   

# (Optional) Find Best Hyperparameters

In [10]:
def initLSTMModel(neurons, dropout_rate, activation):
    model = Sequential()
    model.add(LSTM(seq_len, input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, activation=activation))
    #model.add(Dense(1, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics='mean_absolute_error')
    return model

# Set up parameter grid
param_grid = {
    'neurons': [8, 16],
    #'dropout_rate': [0.2, 0.4],
    'activation': ['relu', 'tanh']
}

model = KerasRegressor(model=initLSTMModel, epochs=16, batch_size=32, neurons=8, dropout_rate=0.2, activation='relu')

# Perform grid search with cross-validation
# scoring='neg_mean_squared_error' as grid search will return maximum scoring
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_result = grid_search.fit(train_x, train_y)

# Evaluate and extract the best model
best_model = grid_result.best_estimator_
test_accuracy = best_model.score(test_x, test_y)
print(best_model)
print(test_accuracy)

Epoch 1/16
239/239 [==============================] - 265s 1s/step - loss: 512.2399 - mean_absolute_error: 17.0619
Epoch 2/16
239/239 [==============================] - 240s 1s/step - loss: 50.8481 - mean_absolute_error: 5.3327
Epoch 3/16
239/239 [==============================] - 250s 1s/step - loss: 36.0855 - mean_absolute_error: 4.5771
Epoch 4/16
239/239 [==============================] - 249s 1s/step - loss: 32.1397 - mean_absolute_error: 4.2620
Epoch 5/16
239/239 [==============================] - 255s 1s/step - loss: 29.2638 - mean_absolute_error: 4.0904
Epoch 6/16
239/239 [==============================] - 258s 1s/step - loss: 27.7472 - mean_absolute_error: 3.9777
Epoch 7/16
239/239 [==============================] - 239s 1s/step - loss: 27.5407 - mean_absolute_error: 3.9412
Epoch 8/16
239/239 [==============================] - 244s 1s/step - loss: 26.1657 - mean_absolute_error: 3.8481
Epoch 9/16
239/239 [==============================] - 240s 1s/step - loss: 26.2449 - mean_abso

# Create Model
### With best hyperparameters

In [ ]:
# Define the model
model = Sequential()
model.add(LSTM(seq_len, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics='mean_absolute_error')

# Train Model

In [ ]:
# Train the model
history = model.fit(train_x, train_y, epochs=8, batch_size=32, validation_data=(test_x, test_y), verbose=True, shuffle=False)

#save the model
model.save('ml2023_model.h5')

# Plot training history
plt.plot(history.history['mean_absolute_error'], label='train')
plt.plot(history.history['val_mean_absolute_error'], label='test')
plt.legend()
plt.show()


In [ ]:
#This part is for testing purpose not really needed
past_10_day_test = pd.read_csv('Past_10_Days.csv')

# remove units from length column and convert to float
past_10_day_test['Timestamp'] = past_10_day_test['Timestamp'].str.replace(' Singapore', '').astype(str)
past_10_day_test['MainIncoming_kW'] = past_10_day_test['MainIncoming_kW'].str.replace('kW', '').astype(float)
past_10_day_test['CHWRT'] = past_10_day_test['CHWRT'].str.replace('°C', '').astype(float)
past_10_day_test['CWRT'] = past_10_day_test['CWRT'].str.replace('°C', '').astype(float)
past_10_day_test['CHWST'] = past_10_day_test['CHWST'].str.replace('°C', '').astype(float)
past_10_day_test['CWST'] = past_10_day_test['CWST'].str.replace('°C', '').astype(float)
past_10_day_test['CHW Flow'] = past_10_day_test['CHW Flow'].str.replace('L/s', '').astype(float)
past_10_day_test['CDW Flow'] = past_10_day_test['CDW Flow'].str.replace('L/s', '').astype(float)
past_10_day_test['heat_in'] = past_10_day_test['heat_in'].str.replace('_RT', '').astype(float)
past_10_day_test['heat_out'] = past_10_day_test['heat_out'].str.replace('_RT', '').astype(float)
past_10_day_test['lift'] = past_10_day_test['lift'].str.replace('Δ°C', '').astype(float)
past_10_day_test['chwdt'] = past_10_day_test['chwdt'].str.replace('Δ°C', '').astype(float)
past_10_day_test['cwdt'] = past_10_day_test['cwdt'].str.replace('Δ°C', '').astype(float)
past_10_day_test['DemoSite Demo avgHum'] = past_10_day_test['DemoSite Demo avgHum'].str.replace('%RH', '').astype(float)
past_10_day_test['DemoSite Demo avgTemp'] = past_10_day_test['DemoSite Demo avgTemp'].str.replace('°C', '').astype(float)
past_10_day_test['DemoSite Demo Calculated Wet Bulb'] = past_10_day_test['DemoSite Demo Calculated Wet Bulb'].str.replace('°C', '').astype(float)

actual_kW= past_10_day_test.loc[287:, ['MainIncoming_kW']]
print(actual_kW)

past_10_day_test['Timestamp'] =pd.to_datetime(past_10_day_test['Timestamp'],dayfirst=True)
# convert the timestamp to the corresponding day of the week
past_10_day_test['Day of Week'] = past_10_day_test['Timestamp'].dt.strftime('%A')
# Convert datetime to weekday numbers (Monday=1, Tuesday=2, etc.)
past_10_day_test['DayNumber'] = past_10_day_test['Timestamp'].dt.weekday
# add 1 to get days starting from 1
past_10_day_test['DayNumber'] = past_10_day_test['DayNumber'] + 1

past_10_day_test['Year'] = past_10_day_test['Timestamp'].dt.year
past_10_day_test['Month'] = past_10_day_test['Timestamp'].dt.month
past_10_day_test['Day'] = past_10_day_test['Timestamp'].dt.day
past_10_day_test['Hour'] = past_10_day_test['Timestamp'].dt.hour
past_10_day_test['Minute'] = past_10_day_test['Timestamp'].dt.minute
past_10_day_test.drop(['Timestamp','MainIncoming_kW','Year','Month','Day','Day of Week'],axis=1, inplace=True)
testing = past_10_day_test

data_arr1=[]
for i in range(len(past_10_day_test)-seq_len+1):
    data_slice = past_10_day_test[i:i+seq_len-1:]
    data_arr1.append(data_slice)
data_arr1 = np.asarray(data_arr1).astype('float32')
past_10_day_test = data_arr1.reshape((data_arr1.shape[0], seq_len-1, data_arr1.shape[2]))
test_Jan_Feb1 = model.predict(past_10_day_test)
test_Jan_Feb1 = pd.DataFrame(test_Jan_Feb1, columns=['Predicted'])
print(test_Jan_Feb1)

test_Jan_Feb1.to_csv("check_test.csv")
actual_kW.to_csv("check_actual.csv")
# Make predictions on test data
#test_pred = model.predict(test_x)

# Calculate MAE and MSE for test and predicted values
test_mae = mean_absolute_error(actual_kW, test_Jan_Feb1)
test_mse = mean_squared_error(actual_kW, test_Jan_Feb1)

print("Test MAE: ", test_mae)
print("Test MSE: ", test_mse)

# Define normalization function
def normalize(values):
    scaler = MinMaxScaler(feature_range=(0, 1))
    normalized = scaler.fit_transform(values.reshape(-1, 1))
    return normalized

#change df to array
test_Jan_Feb_final = test_Jan_Feb1.values
actual_kW = actual_kW.values

# Normalize test_y and predicted_y
actual_kW_norm = normalize(actual_kW)
predicted_kW_norm = normalize(test_Jan_Feb_final)

# Calculate normalized MAE and MSE
norm_mae = np.mean(np.abs(predicted_kW_norm - actual_kW_norm))
norm_mse = np.mean(np.square(predicted_kW_norm - actual_kW_norm))

print("Norm MAE: ", norm_mae)
print("Norm MSE: ", norm_mse)